In [1]:
!pip install earthengine-api crewai psycopg2-binary pandas numpy scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-cohere to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-experimental to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Set OpenAI API Key as environment variable
import os
os.environ['OPENAI_API_KEY'] = 'Provide your key here'  # Replace with your actual OpenAI API key

In [4]:
# Import necessary libraries
import ee
import numpy as np
import psycopg2
from psycopg2 import sql
import crewai
from crewai.agent import Agent
from pydantic import Field, ValidationError

# CrewAI Agent for computing and storing NDVI to PostgreSQL
class Sentinel2ExportAgent(Agent):
    project_id: str = Field(..., description="Project ID for Earth Engine")
    db_url: str = Field(..., description="PostgreSQL URL for Database")

    def __init__(self, name, role, goal, backstory, project_id, db_url):
        try:
            super().__init__(name=name, role=role, goal=goal, backstory=backstory, project_id=project_id, db_url=db_url)
        except ValidationError as e:
            print(e)
            return

        self.project_id = project_id
        self.db_url = db_url
        self.authenticate_and_initialize_earth_engine()

    def authenticate_and_initialize_earth_engine(self):
        try:
            # Authenticate the user
            ee.Authenticate()  # This will prompt OAuth authentication

            # Initialize Earth Engine with the specified project ID
            ee.Initialize(project=self.project_id)
            print(f"Earth Engine initialized with project: {self.project_id}")
        except Exception as e:
            print(f"Error during authentication or initialization: {e}")

    def compute_and_store_ndvi_in_db(self, bands_table_name, ndvi_table_name):
        try:
            # Connect to PostgreSQL database
            conn = psycopg2.connect(self.db_url)
            cursor = conn.cursor()

            # Fetch the B8 and B4 bands from the database
            cursor.execute(sql.SQL(f"SELECT B8, B4 FROM {bands_table_name};"))
            result = cursor.fetchone()

            if result is None:
                print(f"No data found in the {bands_table_name} table.")
                return

            # Fetch B8 and B4 band values from the result
            B8_band = np.array(result[0])  # Assuming B8 is the first column
            B4_band = np.array(result[1])  # Assuming B4 is the second column

            # Compute NDVI: (B8 - B4) / (B8 + B4)
            ndvi = (B8_band - B4_band) / (B8_band + B4_band)

            # Mask out no-data values or invalid NDVI values
            ndvi = np.ma.masked_invalid(ndvi)

            # Store NDVI data in PostgreSQL
            self.insert_ndvi_into_postgres(ndvi_table_name, ndvi)

            # Close the connection
            cursor.close()
            conn.close()

        except Exception as e:
            print(f"Error during NDVI computation or data storage: {e}")

    def insert_ndvi_into_postgres(self, table_name, ndvi_data):
        try:
            # Connect to PostgreSQL database
            conn = psycopg2.connect(self.db_url)
            cursor = conn.cursor()

            # Create NDVI table if it doesn't exist
            cursor.execute(sql.SQL(f"""
                CREATE TABLE IF NOT EXISTS {table_name} (
                    id SERIAL PRIMARY KEY,
                    NDVI FLOAT[]
                );
            """))

            # Insert NDVI data into the table
            cursor.execute(sql.SQL(f"""
                INSERT INTO {table_name} (NDVI)
                VALUES (%s);
            """), (ndvi_data.tolist(),))

            # Commit and close the connection
            conn.commit()
            cursor.close()
            conn.close()

            print(f"NDVI data inserted into table '{table_name}'.")

        except Exception as e:
            print(f"Error inserting NDVI data into PostgreSQL: {e}")

In [5]:
# Main code execution
if __name__ == "__main__":
    # Set up project details and database URL
    project_id = 'Provide your project ID here'  # Replace with your actual project ID
    db_url = 'Provide your db url here'
    # Define the role, goal, and backstory for the agent
    role = "Data Analyst"
    goal = "Fetch bands from PostgreSQL, compute NDVI, and store NDVI back in PostgreSQL"
    backstory = "The agent assists in data analysis by fetching bands from PostgreSQL and computing NDVI for further analysis."

    # Create and run the CrewAI agent
    sentinel2_agent = Sentinel2ExportAgent(
        name="Sentinel2ExportAgent",
        role=role,
        goal=goal,
        backstory=backstory,
        project_id=project_id,
        db_url=db_url
    )

    # Example usage: fetch B8 and B4 bands, compute NDVI, and store it in PostgreSQL
    bands_table_name = 'sentinel2_bands_data'  # Table storing the bands data
    ndvi_table_name = 'ndvi_data'  # Table to store the computed NDVI

    sentinel2_agent.compute_and_store_ndvi_in_db(bands_table_name, ndvi_table_name)


Earth Engine initialized with project: genai-agent-hack-2024
NDVI data inserted into table 'ndvi_data'.
